In [17]:
import re
import requests 
import pandas as pd


###Lugares Turisticos de Centro america y sudamerica
consulta1 = requests.get('https://www.aarp.org/espanol/turismo/internacional/info-2016/fotos-patrimonios-culturales-america-latina.html')
consulta2 = requests.get('https://viajerocasual.com/lugares-turisticos-de-sudamerica/')

lugares1 = re.findall(r'<h2 id="listTitle.*?">(.*?)</h2>' , consulta1.text , re.DOTALL)
lugares2 = re.findall(r'<h2 class="wp-block-heading"><span class="ez-toc-section" id="[^"]*"></span>(.*?)<span class="ez-toc-section-end"></span></h2>' , 
                     consulta2.text , re.DOTALL)
Lugares = []
Paises = []

for i in lugares1:
    Lugares.append(i.split(',')[0].strip())
    Paises.append(i.split(',')[1].strip())
    
for i in lugares2:
    Lugares.append(i.split(',')[0].strip())
    Paises.append(i.split(',')[1].strip())

Datos = pd.DataFrame({'Lugares':Lugares , 'Paises':Paises})
Datos





,Lugares,Paises
0,Conjunto arquitectónico de Pampulha,Brasil
1,Castillo de San Pedro de la Roca,Santiago de Cuba
2,Cueva de las manos,Argentina
3,Tiwanaku,Bolivia
4,Paisaje cultural cafetero,Colombia
...,...,...
58,Cabo Polonio,Uruguay
59,Ostrov Navarino,Chile
60,Catedral de Sal,Zipaquirá
61,Mindo,Ecuador


In [18]:
### Añadiendo coordenadas
from geopy.geocoders import Nominatim

def obtener_coordenadas(nombre_lugar):
    geolocator = Nominatim(user_agent="mi_app_de_mapas")
    location = geolocator.geocode(nombre_lugar)
    if location:
        return location.latitude, location.longitude
    else:
        return None


coordenadas = []
for i in range(len(Lugares)):
    xy = str(Lugares[i]) + str(',') + str(' ') +str(Paises[i])
    rest = obtener_coordenadas(xy)
    coordenadas.append(rest)
    
# añadiendo a la tabla

Datos['Coordenadas'] = coordenadas

Datos



,Lugares,Paises,Coordenadas
0,Conjunto arquitectónico de Pampulha,Brasil,None
1,Castillo de San Pedro de la Roca,Santiago de Cuba,"(19.968284099999998, -75.86991363552517)"
2,Cueva de las manos,Argentina,"(-47.1552731, -70.6559679)"
3,Tiwanaku,Bolivia,"(-16.4994445, -68.1243969)"
4,Paisaje cultural cafetero,Colombia,"(4.9828018, -75.6056022)"
...,...,...,...
58,Cabo Polonio,Uruguay,"(-34.4028524, -53.7825007)"
59,Ostrov Navarino,Chile,"(-55.1122988, -67.60622943544601)"
60,Catedral de Sal,Zipaquirá,"(5.0187088, -74.0103258)"
61,Mindo,Ecuador,"(-0.0529382, -78.7746504)"


In [19]:
#Limpiando 
Datos = Datos.dropna(subset=['Coordenadas'])
Datos = Datos.reset_index(drop=True)
Datos


,Lugares,Paises,Coordenadas
0,Castillo de San Pedro de la Roca,Santiago de Cuba,"(19.968284099999998, -75.86991363552517)"
1,Cueva de las manos,Argentina,"(-47.1552731, -70.6559679)"
2,Tiwanaku,Bolivia,"(-16.4994445, -68.1243969)"
3,Paisaje cultural cafetero,Colombia,"(4.9828018, -75.6056022)"
4,Ciudad de Cuzco,Perú,"(-13.5169353, -71.97927381639681)"
5,Parque Nacional Tikal,Guatemala,"(17.1910467, -89.61489111940308)"
6,Reserva de la Biósfera de la Mariposa Monarca,México,"(19.5241105, -100.20477526215062)"
7,Ciudad Colonial de Santo Domingo,República Dominicana,"(18.4714798, -69.8868222)"
8,Parque Nacional Torres del Paine,Chile,"(-51.0405776, -72.98465008755775)"
9,Salvador,Brasil,"(-12.9822499, -38.4812772)"


In [50]:



Temperaturas = []
Humedad = []

for i in range(Datos.shape[0]):
    coordenada = str(Datos.iloc[i]['Coordenadas'])
    lat, lon = coordenada.strip('()').split(',')
    lat = float(lat)
    lon = float(lon)
    api_key = 'cf62addae9c1d0360bf97aa196222a0b'
    url = f'http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}'
    
    consulta = requests.get(url)
    consulta = consulta.json()
    temperature = round(consulta['main']['temp'] - 273.15, 2)
    humidity = consulta['main']['humidity']
    
    Temperaturas.append(temperature)
    Humedad.append(humidity)

Datos['Temperaturas'] = Temperaturas
Datos['Humedad'] = Humedad

Datos




,Lugares,Paises,Coordenadas,Temperaturas,Humedad
0,Castillo de San Pedro de la Roca,Santiago de Cuba,"(19.968284099999998, -75.86991363552517)",30.14,84
1,Cueva de las manos,Argentina,"(-47.1552731, -70.6559679)",2.01,77
2,Tiwanaku,Bolivia,"(-16.4994445, -68.1243969)",12.96,23
3,Paisaje cultural cafetero,Colombia,"(4.9828018, -75.6056022)",16.89,100
4,Ciudad de Cuzco,Perú,"(-13.5169353, -71.97927381639681)",16.13,36
5,Parque Nacional Tikal,Guatemala,"(17.1910467, -89.61489111940308)",26.45,79
6,Reserva de la Biósfera de la Mariposa Monarca,México,"(19.5241105, -100.20477526215062)",14.22,92
7,Ciudad Colonial de Santo Domingo,República Dominicana,"(18.4714798, -69.8868222)",28.91,84
8,Parque Nacional Torres del Paine,Chile,"(-51.0405776, -72.98465008755775)",0.27,92
9,Salvador,Brasil,"(-12.9822499, -38.4812772)",24.85,78


In [60]:
#Cercania: 

def distancia(Ubicacion_usuario, Destino):
    Coordenadas_usuario = str(obtener_coordenadas(Ubicacion_usuario))
    latU, lonU = coordenada.strip('()').split(',')
    latD, lonD = Destino.strip('()').split(',')
    latU= float(latU)
    latD= float(latD)
    lonU= float(lonU)
    lonD= float(lonD)
    distancia = (((latU-latD)**2)+((lonU-lonD)**2))**(1/2)
    return round(distancia, 2)

distancia('Lima, Peru' , '(-55.1122988, -67.60622943544601)')


    


    

51.67